<a href="https://colab.research.google.com/github/abzb1/UoS_AI_20026/blob/main/03_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U uv
!uv pip install -U unsloth vllm==0.10.1 numpy==2.0.2 torchvision bitsandbytes xformers transformers
!uv pip install triton==3.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 66.9 MB/s eta 0:00:00
Using Python 3.12.11 environment at: /usr
Resolved 165 packages in 1.51s
Prepared 76 packages in 1m 11s
Uninstalled 33 packages in 1.37s
Installed 76 packages in 697ms
 - accelerate==1.10.0
 + accelerate==1.10.1
 + astor==0.8.1
 + bitsandbytes==0.47.0
 + blake3==1.0.5
 - cachetools==5.5.2
 + cachetools==6.2.0
 + cbor2==5.7.0
 + compressed-tensors==0.10.2
 - cupy-cuda12x==13.3.0
 + cupy-cuda12x==13.6.0
 + cut-cross-entropy==25.1.1
 - datasets==4.0.0
 + datasets==3.6.0
 + depyf==0.19.0
 - diffusers==0.34.0
 + diffusers==0.35.1
 + diskcache==5.6.3
 + dnspython==2.7.0
 + email-validator==2.3.0
 + fastapi-cli==0.0.8
 + fastapi-cloud-cli==0.1.5
 + gguf==0.17.1
 - hf-xet==1.1.7
 + hf-xet==1.1.8
 + httptools==0.6.4
 + interegular==0.3.3
 + lark==1.2.2
 + llguidance==0.7.30
 - llvmlite==0.43.0
 + llvmlite==0.44.0
 + lm-format-enforcer==0.10.12
 + mistral-common==1.8.4
 + msgspec==0.19.0
 + ninja==1.13.0
 - numba==0.6

In [2]:
from unsloth import FastLanguageModel

import os
import re
import math

import numpy as np
import pandas as pd

from transformers import TextStreamer
from datasets import Dataset, load_dataset
from trl import SFTTrainer, SFTConfig, GRPOConfig, GRPOTrainer

from vllm import SamplingParams

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 08-26 14:18:57 [__init__.py:241] Automatically detected platform cuda.
ERROR 08-26 14:18:59 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# load model and tokenizer

SEED = 1784
MODEL_NAME = "unsloth/Qwen3-4B-Base"

MAX_SEQ_LEN = 2048
LORA_RANK   = 16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LEN,
    load_in_4bit=False,
    fast_inference=True,
    max_lora_rank=LORA_RANK,
    gpu_memory_utilization=0.7,
)

Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.9: Fast Qwen3 patching. Transformers: 4.55.4. vLLM: 0.10.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.33 GB. Also swap space = 0 GB.
Unsloth: Not an error, but `device` is not supported in vLLM. Skipping.
INFO 08-26 14:19:15 [utils.py:326] non-default args: {'model': 'unsl

INFO 08-26 14:19:50 [cuda.py:384] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 08-26 14:19:50 [cuda.py:433] Using XFormers backend.
INFO 08-26 14:19:51 [parallel_state.py:1134] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
INFO 08-26 14:19:51 [model_runner.py:1080] Starting to load model unsloth/Qwen3-4B-Base...
INFO 08-26 14:19:52 [weight_utils.py:296] Using model weights format ['*.safetensors']


INFO 08-26 14:21:40 [weight_utils.py:312] Time spent downloading weights for unsloth/Qwen3-4B-Base: 107.782608 seconds


INFO 08-26 14:22:16 [default_loader.py:262] Loading weights took 36.02 seconds
INFO 08-26 14:22:16 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 08-26 14:22:19 [model_runner.py:1112] Model loading took 7.5719 GiB and 144.679509 seconds
INFO 08-26 14:22:38 [worker.py:295] Memory profiling takes 17.87 seconds
INFO 08-26 14:22:38 [worker.py:295] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.69) = 10.22GiB
INFO 08-26 14:22:38 [worker.py:295] model weights take 7.57GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.88GiB; the rest of the memory reserved for KV Cache is 1.74GiB.
INFO 08-26 14:22:40 [executor_base.py:114] # cuda blocks: 792, # CPU blocks: 0
INFO 08-26 14:22:40 [executor_base.py:119] Maximum concurrency for 2048 tokens per request: 6.19x
INFO 08-26 14:22:40 [vllm_utils.py:671] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 08-26 14:22:40 [model_runner.py:1383] Capturing cudagraphs for decoding.

INFO 08-26 14:23:03 [model_runner.py:1535] Graph capturing finished in 23 secs, took 0.27 GiB
INFO 08-26 14:23:03 [vllm_utils.py:678] Unsloth: Patched vLLM v0 graph capture finished in 23 secs.
INFO 08-26 14:23:04 [llm_engine.py:417] init engine (profile, create kv cache, warmup model) took 44.77 seconds
INFO 08-26 14:23:04 [llm.py:298] Supported_tasks: ['generate']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']


In [4]:
# load LoRA

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=LORA_RANK * 2,
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
)

Unsloth 2025.8.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [5]:
# set chat template and test tokenizer

REASONING_START = "<start_working_out>"
REASONING_END   = "<end_working_out>"
SOLUTION_START  = "<SOLUTION>"
SOLUTION_END    = "</SOLUTION>"

SYSTEM_PROMPT = (
    f"You are given a problem.\n"
    f"Think about the problem and provide your working out.\n"
    f"Place it between {REASONING_START} and {REASONING_END}.\n"
    f"Then, provide your solution between {SOLUTION_START}{SOLUTION_END}"
)

tokenizer.chat_template = (
    "{% if messages and messages[0]['role']=='system' %}"
    "{{ messages[0]['content'] + eos_token }}"
    "{% set loop_messages = messages[1:] %}"
    "{% else %}"
    "{{ '" + SYSTEM_PROMPT + "' + eos_token }}"
    "{% set loop_messages = messages %}"
    "{% endif %}"
    "{% for message in loop_messages %}"
    "{% if message['role']=='user' %}{{ message['content'] }}"
    "{% elif message['role']=='assistant' %}{{ message['content'] + eos_token }}"
    "{% endif %}{% endfor %}"
    "{% if add_generation_prompt %}{{ '" + REASONING_START + "' }}{% endif %}"
)

print(tokenizer.apply_chat_template(
    [
        {"role" : "user", "content" : "What is 1+1?"},
        {"role" : "assistant", "content" : f"{REASONING_START}I think it's 2.{REASONING_END}{SOLUTION_START}2{SOLUTION_END}"},
        {"role" : "user", "content" : "What is 2+2?"},
    ],
    tokenize = False,
    add_generation_prompt = True)
)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|endoftext|>What is 2+2?<start_working_out>


In [6]:
# prepare sft dataset

sft_ds = load_dataset("unsloth/OpenMathReasoning-mini", split="cot").to_pandas()
sft_ds = sft_ds[["expected_answer", "problem", "generated_solution"]]

is_number = np.array(pd.to_numeric(pd.Series(sft_ds["expected_answer"]), errors="coerce").notnull())
sft_ds = sft_ds.iloc[np.where(is_number)[0]].copy()

def to_messages(row):
    thoughts = str(row["generated_solution"]).replace("<think>", "").replace("</think>", "").strip()
    final = (
        REASONING_START + thoughts + REASONING_END +
        SOLUTION_START + str(row["expected_answer"]) + SOLUTION_END
    )
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": str(row["problem"])},
        {"role": "assistant", "content": final},
    ]

sft_ds["Messages"] = sft_ds.apply(to_messages, axis=1)
sft_ds["tok_len"] = sft_ds["Messages"].apply(lambda m: len(tokenizer.apply_chat_template(m)))
sft_ds = sft_ds.loc[sft_ds["tok_len"] <= MAX_SEQ_LEN // 2].copy()

sft_ds["text"] = tokenizer.apply_chat_template(sft_ds["Messages"].tolist(), tokenize=False)
sft_ds = Dataset.from_pandas(sft_ds[["text", "Messages"]])

In [7]:
# do sft

args = SFTConfig(
    dataset_text_field="text",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    warmup_steps=5,
    num_train_epochs=1,
    learning_rate=2e-4,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    logging_steps=5,
    seed=SEED,
    report_to="none",
    bf16=False,
)
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=sft_ds,
    args=args,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 59 | Num Epochs = 1 | Total steps = 59
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Step,Training Loss
5,0.656600
10,0.676900
15,0.442000
20,0.395900
25,0.427200
30,0.451500
35,0.482400
40,0.423400
45,0.446700
50,0.354700


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=59, training_loss=0.46986434621325995, metrics={'train_runtime': 90.4538, 'train_samples_per_second': 0.652, 'train_steps_per_second': 0.652, 'total_flos': 1176498023377920.0, 'train_loss': 0.46986434621325995})

In [8]:
# test model after sft

text = tokenizer.apply_chat_template(
    sft_ds[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True,
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Jenifer has 82 cents in pennies and nickels. Her younger brother mistook all her nickels for dimes and counted the total as $1.47. How many pennies does Jenifer have?<start_working_out>Okay, let's see. Jenifer has 82 cents in pennies and nickels. Her younger brother thought all her nickels were dimes and counted the total as $1.47. I need to find out how many pennies Jenifer has.

First, I need to set up some equations. Let's say the number of nickels Jenifer has is N, and the number of pennies is P. Since nickels are worth 5 cents and pennies are worth 1 cent, the total value is 5N + P = 82 cents.

But her brother thought all the nickels were dimes. A dime is worth 10 cents. So if he counted all the nickels as dimes, the total would be 10N + P = 147 cents (since $1.47 is 147 cent

In [9]:
# prepare grpo dataset

grpo_ds = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split="train")

def map_item(x):
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user",   "content": x["prompt"]},
        ],
        "answer": x["solution"],
    }
grpo_ds = grpo_ds.map(map_item)

tokenized = grpo_ds.map(lambda ex: {
    "tokens": tokenizer.apply_chat_template(ex["prompt"], add_generation_prompt=True, tokenize=True)
})
lengths = np.array([len(t) for t in tokenized["tokens"]])
max_prompt_len = int(np.quantile(lengths, 0.9))
keep_idx = np.where(lengths <= max_prompt_len)[0]
grpo_ds = grpo_ds.select(keep_idx)

In [10]:
# set GRPO configs

GRPO_STEPS        = 10
GRPO_LR           = 5e-5
GRPO_WD           = 0.01
GRPO_WARMUP_RATIO = 0.1
GRPO_NUM_GEN      = 4
TEMP              = 1.0

max_prompt_len = int(max_prompt_len) + 1
max_completion_len = MAX_SEQ_LEN - max_prompt_len

sampling = SamplingParams(
    min_p=0.1, top_p=1.0, top_k=-1,
    seed=SEED, stop=[tokenizer.eos_token], include_stop_str_in_output=True
)
cfg = GRPOConfig(
    vllm_sampling_params=sampling,
    temperature=TEMP,
    learning_rate=GRPO_LR,
    weight_decay=GRPO_WD,
    warmup_ratio=GRPO_WARMUP_RATIO,
    lr_scheduler_type="linear",
    optim="adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_generations=GRPO_NUM_GEN,
    max_prompt_length=max_prompt_len,
    max_completion_length=max_completion_len,
    max_steps=GRPO_STEPS,
    save_steps=GRPO_STEPS,
    report_to="none",
    output_dir="outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [11]:
# def GRPO reward function

SOL_END_REGEX = r"</SOLUTION>\s*(?:" + re.escape("</s>") + r")?"
MATCH_FORMAT = re.compile(
    rf"{re.escape(REASONING_END)}.*?{re.escape(SOLUTION_START)}(.+?){SOL_END_REGEX}\s*$",
    flags=re.DOTALL | re.MULTILINE,
)
NUM_IN_SOLUTION = re.compile(
    re.escape(SOLUTION_START) + r".*?([-]?[\d\.,]+)",
    flags=re.DOTALL | re.MULTILINE,
)

def _extract_solution_text(response: str):
    m = MATCH_FORMAT.search(response)
    return m.group(1) if m else None

def _coerce_float(x: str):
    try:
        return float(x.strip().replace(",", ""))
    except Exception:
        return None

def reward_match_format_exact(completions, **_):
    scores = []
    for c in completions:
        r = c[0]["content"]
        scores.append(3.0 if MATCH_FORMAT.search(r) else 0.0)
    return scores

def reward_match_format_approx(completions, **_):
    scores = []
    for c in completions:
        r = c[0]["content"]
        s = 0.0
        s += 0.5 if r.count(REASONING_END)  == 1 else -1.0
        s += 0.5 if r.count(SOLUTION_START) == 1 else -1.0
        s += 0.5 if r.count(SOLUTION_END)   == 1 else -1.0
        scores.append(s)
    return scores

def reward_answer_exact(prompts, completions, answer, **_):
    scores = []
    extracted = [_extract_solution_text(c[0]["content"]) for c in completions]
    for g, t in zip(extracted, answer):
        if g is None:
            scores.append(-2.0); continue
        if g == t:
            scores.append(5.0)
        elif g.strip() == str(t).strip():
            scores.append(3.5)
        else:
            gf, tf = _coerce_float(g), _coerce_float(str(t))
            if gf is None or tf is None:
                scores.append(-2.0)
            else:
                ratio = gf / (tf + 1e-9)
                if 0.9 <= ratio <= 1.1: scores.append(2.0)
                elif 0.8 <= ratio <= 1.2: scores.append(1.5)
                else: scores.append(-2.5)
    return scores

def reward_answer_numeric(prompts, completions, answer, **_):
    scores = []
    for c, t in zip(completions, answer):
        r = c[0]["content"]
        m = NUM_IN_SOLUTION.search(r)
        if not m:
            scores.append(-2.5); continue
        gf = _coerce_float(m.group(1))
        tf = _coerce_float(str(t))
        if gf is None or tf is None:
            scores.append(0.0)
        else:
            scores.append(3.5 if math.isclose(gf, tf, rel_tol=0, abs_tol=0) else -1.5)
    return scores

In [12]:
# do GRPO

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        reward_match_format_exact,
        reward_match_format_approx,
        reward_answer_exact,
        reward_answer_numeric,
    ],
    args=cfg,
    train_dataset=grpo_ds,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,709 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,entropy,rewards / reward_match_format_exact / mean,rewards / reward_match_format_exact / std,rewards / reward_match_format_approx / mean,rewards / reward_match_format_approx / std,rewards / reward_answer_exact / mean,rewards / reward_answer_exact / std,rewards / reward_answer_numeric / mean,rewards / reward_answer_numeric / std
1,0.000100,-7.500000,0.000000,1844.000000,1844.000000,1844.000000,1.000000,0.000000,0.000000,0.000000,0.077926,0,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-2.500000,0.000000
2,0.000100,-2.500000,3.829709,1573.750000,1295.000000,1844.000000,0.250000,1483.666748,1295.000000,1746.000000,0.089135,No Log,1.500000,1.732051,0.000000,2.121320,-2.250000,0.288675,-1.750000,0.500000
3,0.000100,-2.375000,6.196437,1467.500000,942.000000,1844.000000,0.500000,1091.000000,942.000000,1240.000000,0.114402,No Log,1.500000,1.732051,-0.750000,2.598076,-1.125000,2.096624,-2.000000,0.577350
4,0.000100,-7.500000,0.000000,1844.000000,1844.000000,1844.000000,1.000000,0.000000,0.000000,0.000000,0.081220,No Log,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-2.500000,0.000000
5,0.000100,13.000000,0.000000,1199.500000,1076.000000,1293.000000,0.000000,1199.500000,1076.000000,1293.000000,0.096097,No Log,3.000000,0.000000,1.500000,0.000000,5.000000,0.000000,3.500000,0.000000
6,0.000100,-7.500000,0.000000,1775.500000,1570.000000,1844.000000,0.750000,1570.000000,1570.000000,1570.000000,0.069634,No Log,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-2.500000,0.000000
7,0.000100,-0.500000,2.000000,1305.000000,777.000000,1547.000000,0.000000,1305.000000,777.000000,1547.000000,0.097207,No Log,2.250000,1.500000,1.125000,0.750000,-2.375000,0.250000,-1.500000,0.000000
8,0.000100,2.750000,11.835680,1413.500000,978.000000,1844.000000,0.500000,983.000000,978.000000,988.000000,0.084516,No Log,1.500000,1.732051,-0.750000,2.598076,1.500000,4.041452,0.500000,3.464102
9,0.000100,-7.500000,0.000000,1844.000000,1844.000000,1844.000000,1.000000,0.000000,0.000000,0.000000,0.090350,No Log,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-2.500000,0.000000
10,0.000100,4.750000,10.070584,1476.750000,1039.000000,1844.000000,0.250000,1354.333374,1039.000000,1812.000000,0.082037,No Log,2.250000,1.500000,0.375000,2.250000,1.375000,4.190763,0.750000,3.201562


TrainOutput(global_step=10, training_loss=8.825896293274127e-05, metrics={'train_runtime': 1036.0188, 'train_samples_per_second': 0.039, 'train_steps_per_second': 0.01, 'total_flos': 0.0, 'train_loss': 8.825896293274127e-05})

In [13]:
# test after GRPO

text = "What is the sqrt of 101?"

sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

print(output)

 How is the sqrt of 101 represented? What is the square root of 101?

The square root of 101, denoted as \(\sqrt{101}\), is approximately 10.04987562112090 when calculated to twelve decimal places. The square root of 101 can be represented as \(\sqrt{101}\) or \(101^{1/2}\). To find the square root of 101, we look for a number that, when multiplied by itself, results in 101. Since 101 is not a perfect square, its square root is an irrational number, meaning it cannot be expressed as a simple fraction and has a non-repeating, non-terminating decimal expansion.


In [14]:
# save model

save_dir = "grpo_saved_lora"
model.save_lora(save_dir)